# Remora Risk Score vs Price Visualization

This notebook demonstrates how Remora's risk score correlates with price movements and market conditions.

**Key insights:**
- High risk scores often coincide with volatility spikes
- Remora blocks entries during dangerous periods
- Visual correlation between risk and price action


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from remora.client import RemoraClient

# Initialize Remora client
client = RemoraClient()


## Fetch Historical Risk Data

For this example, we'll simulate fetching risk data over time. In practice, you would:
1. Run backtests with Remora enabled
2. Extract risk scores from your strategy logs
3. Compare against price data


In [ ]:
# Example: Get current risk context for BTC/USDT
symbol = "BTC/USDT"
ctx = client.get_context(symbol)

print(f"Symbol: {symbol}")
print(f"Safe to trade: {ctx.get('safe_to_trade')}")
print(f"Risk score: {ctx.get('risk_score'):.2f}")
print(f"Risk class: {ctx.get('risk_class', 'N/A')}")
print(f"Reasoning: {ctx.get('reasoning', [])}")


## Simulated Risk vs Price Plot

**Note:** This is a conceptual example. In a real scenario, you would:
- Fetch historical price data from your exchange
- Get risk scores from your backtest results or historical API calls
- Plot them together to see correlation

The visualization below shows what such an analysis would look like.


In [ ]:
# Simulated data for demonstration
# In practice, use real price and risk data from your backtests
import numpy as np

# Generate sample timestamps (last 7 days, hourly)
dates = pd.date_range(end=datetime.now(), periods=168, freq='H')

# Simulated price data (BTC-like movement)
np.random.seed(42)
price_base = 45000
price_data = price_base + np.cumsum(np.random.randn(168) * 100)

# Simulated risk scores (higher during volatility)
volatility = np.abs(np.random.randn(168))
risk_scores = np.clip(volatility / 3, 0, 1)

# Mark high-risk periods (where Remora would block entries)
blocked_periods = risk_scores > 0.5

# Create DataFrame
df = pd.DataFrame({
    'timestamp': dates,
    'price': price_data,
    'risk_score': risk_scores,
    'blocked': blocked_periods
})


In [ ]:
# Create visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot 1: Price with blocked entry zones highlighted
ax1.plot(df['timestamp'], df['price'], label='Price', color='#2E86AB', linewidth=1.5)
ax1.fill_between(df['timestamp'], df['price'].min(), df['price'].max(), 
                 where=df['blocked'], alpha=0.3, color='red', label='Remora Blocked Entries')
ax1.set_ylabel('Price (USDT)', fontsize=12)
ax1.set_title('Price Action with Remora Risk Zones', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Risk score
ax2.plot(df['timestamp'], df['risk_score'], label='Risk Score', color='#A23B72', linewidth=1.5)
ax2.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='High Risk Threshold')
ax2.fill_between(df['timestamp'], 0, df['risk_score'], alpha=0.3, color='#A23B72')
ax2.set_ylabel('Risk Score (0-1)', fontsize=12)
ax2.set_xlabel('Time', fontsize=12)
ax2.set_title('Remora Risk Score Over Time', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*60)
print("Summary Statistics")
print("="*60)
print(f"Total periods analyzed: {len(df)}")
print(f"High-risk periods (blocked): {df['blocked'].sum()} ({df['blocked'].sum()/len(df)*100:.1f}%)")
print(f"Average risk score: {df['risk_score'].mean():.3f}")
print(f"Max risk score: {df['risk_score'].max():.3f}")
print(f"Price volatility (std): ${df['price'].std():.2f}")


## Key Takeaways

1. **Red zones** show when Remora would block entries
2. **Risk spikes** often correlate with price volatility
3. **Fewer entries** during high-risk periods = fewer losing trades
4. **Smoother equity curve** by avoiding dangerous market conditions

## Next Steps

- Run actual backtests with `simple_risk_filter_strategy` or `advanced_risk_filter_strategy`
- Export risk scores from your strategy logs
- Compare win rates before/after Remora filtering
- See the backtesting repository for real comparisons: https://github.com/DonaldSimpson/remora-backtests
